In [1]:
import sys
import os
import time
from typing import List, Dict
import json
from uuid import uuid4

import pandas as pd

from tools.chunker import Chunker
from tools.embedding import EmbeddingService
from n4j.communicator import Communicator

from utils.utils import batch_method, prepare_new_nodes, remove_filler_words


## Define variables

In [2]:
with open("resources/playlist_ids.json") as json_file:
    playlists = json.load(json_file)

In [3]:
list(playlists.keys())[0]

'classics'

In [4]:
chunker = Chunker()

In [5]:
embed = EmbeddingService()

In [6]:
writer = Communicator()

driver created


In [7]:
playlist = 'classics'
playlist_id = playlists[playlist]

## Load Code

In [8]:
try:
    writer.create_constraints()
except Exception as e:
    print(e)

database name:  neo4j
{code: Neo.ClientError.Schema.EquivalentSchemaRuleAlreadyExists} {message: An equivalent constraint already exists, 'Constraint( id=3, name='document_id', type='UNIQUENESS', schema=(:Document {id}), ownedIndex=2 )'.}


In [9]:
chunker._chunked_documents = []
chunker.chunk_youtube_transcripts(playlist_title=playlist,
                                cleaning_functions=[remove_filler_words])

In [10]:
chunker.chunks_as_list[0]

{'url': 'https://www.youtube.com/watch?v=-7_UEorW3Vg',
 'video_id': '-7_UEorW3Vg',
 'parent_index': 'dce16fc2-f756-4f05-9ad1-74a67735ce54',
 'parent_transcript': "hi everyone Anthony Fantino here Internet's busiest music nerd Happy New Year and it's time for a nother review of an older album today Dead Kennedys plastic surgery disasters Dead Kennedys is a California hardcore punk band who at the time of the release of this album in the early 80s their roster consisted of jello biafra on vocals D H peligro on drums Klaus flouride on bass and East Bay Ray on guitar and in my opinion this was one of the best quartets in hardcore punk during their time I mean tight talented unique and very creative this help he is Dead Kennedys sophomore full-length for the most part I've seen more I guess sort of accolade it's given to their debut album people really prefer that LP over this one but in my opinion this album is actually nothing but improvement past the debut LP maybe it didn't really have 

In [11]:
result = []
failed_idx = None
playlist_total = len(chunker.chunks_as_list)

In [12]:
print(playlist+": "+str(playlist_total))
print("grabbing embeddings...")

for idx, bat in enumerate(batch_method(chunker.chunks_as_list, 20)):
    new_nodes = prepare_new_nodes(data=bat, playlist_id=playlist_id, embedding_service=embed)
    result+=new_nodes
    print("total percent: ", str(round(((20*idx)+1) / playlist_total, 4)*100)[:4], "%", " batch", idx+1, end="\r")

classics: 6897
grabbing embeddings...


In [13]:
print("loading to graph...")
writer.load_nodes(data=result)

loading to graph...
database name:  neo4j
